In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Utils
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, roc_auc_score, make_scorer, accuracy_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import StratifiedKFold
import lightgbm
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_uniform
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise  import cosine_similarity
from sklearn.metrics.pairwise import chi2_kernel
from catboost import CatBoostClassifier
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
from sklearn.manifold import TSNE
from sklearn.ensemble import IsolationForest
from sklearn.metrics import log_loss
from sklearn.preprocessing import KBinsDiscretizer
import category_encoders as ce
from tpot import TPOTClassifier
import timeit
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from tpot.builtins import ZeroCount
from sklearn.preprocessing import RobustScaler

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Load Data

In [ ]:
train= pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv', sep=',')
test= pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv', sep=',')

In [ ]:
train = train.set_index('id')
test = test.set_index('id')

In [ ]:
sub_sample = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv', sep=',')
sub_sample = sub_sample.set_index('id')

In [ ]:
fig = px.histogram(train, x="target",
                   width=600, 
                   height=400,
                   histnorm='percent',
                   template="simple_white"
                   )
fig.update_layout(title="Target Description", 
                  font_family="San Serif",
                  titlefont={'size': 20},
                  showlegend=True,
                  legend=dict(
                      orientation="v", 
                      y=1, 
                      yanchor="top", 
                      x=1.0, 
                      xanchor="right"
                  )                
                 ).update_xaxes(categoryorder='total descending')#
fig.update_traces( 
                  marker_line_width=1.5, opacity=0.99)
fig.show()

### Estimation model Log-loss vs Random Guessing

In [ ]:
def calc_loss(class_perc, num):
    
    lst=[]
    
    for i,z in enumerate(class_perc):
        lst = lst+[i for x in range(int(z*(num+1)))]
        
    preds=[]
    
    for i in range(num):
        preds+=[class_perc]
    
    return (log_loss(lst,preds))

In [ ]:
#example of how could be the distribution in classes according to the LB
calc_loss([0.60, 0.20, 0.10, 0.10], 200000)

In [ ]:
#target Map
dict1 = dict(zip(list(train.target.unique()),range(4)))
dict1

In [ ]:
train['target']= train['target'].replace(dict1)

In [ ]:
cols = train.columns[:-1]
target = train['target']
train = train[cols]

In [ ]:
train[train<0]=0
test[test<0]=0

In [ ]:
# create train and test sets
X_train, X_val, y_train, y_val = train_test_split(train, target, train_size=0.75, test_size=0.25, stratify=target, random_state=2)

### TPOT Classifier

In [ ]:
tpot = TPOTClassifier(verbosity=3, 
                      scoring='neg_log_loss', 
                      random_state=2, 
                      periodic_checkpoint_folder='tpot_tpsmay.txt', 
                      n_jobs=-1,
                      cv=5,
                      generations=5, 
                      population_size=10)

In [ ]:
start_time = timeit.default_timer()
tpot.fit(X_train, y_train)
winning = tpot.fitted_pipeline_
score = (tpot.score(X_val, y_val))
tpot.export('tps_may_pipeline.py')  
print('Winning pipeline :', winning, 'Score:', score)

In [ ]:
print('Winning pipeline :', winning, 'Score:', score)

### Train the Model

In [ ]:
preds = np.zeros((len(test), 4))
oof_preds = pd.DataFrame(index=train.index)
k=10
kf = StratifiedKFold(n_splits = k, random_state = 22 , shuffle = True)
roc = []

n = 0
for trn_idx , val_idx in kf.split(train , target):
    train_x = train.iloc[trn_idx]
    train_y = target.iloc[trn_idx]
    val_x = train.iloc[val_idx]
    val_y = target.iloc[val_idx]
    
    
   
    
    
    model = Pipeline(steps=[('zerocount', ZeroCount()), ('robustscaler', RobustScaler()),
                ('mlpclassifier',
                 MLPClassifier(alpha=0.1, learning_rate_init=0.01,
                               random_state=2))])
    
    
    if n!=0:
        model.fit(train_x , train_y)
        preds += model.predict_proba(test)
        preds0 = preds/(k)
        roc.append(log_loss(val_y ,model.predict_proba(val_x)))
        print(n+1 , roc[n])

    if n==0:
        roc.append(0)
        print(n+1 , roc[n])
        
    
    n+=1

In [ ]:
p = pd.DataFrame(preds0, index=test.index)

In [ ]:
p.head()

In [ ]:
sub_sample['Class_1']=p[1]
sub_sample['Class_2']=p[0]
sub_sample['Class_3']=p[3]
sub_sample['Class_4']=p[2]

In [ ]:
sub_sample = sub_sample.reset_index()
sub_sample.head()

In [ ]:
sub_sample.to_csv('submission.csv',index=False)